In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import os
import pickle

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """load data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding = 'latin')
        return data['data'], data['labels']
    
class CifarData:
    """定义一个类处理cifar的数据"""
    def __init__(self, filenames, need_shuffle): #对于训练集来说需要shuffle，打乱数据以增加训练成效，对于测试集来说就不需要。
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename) 
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data) #纵向组合成一个矩阵
        self._data = self._data / 127.5 - 1   
        self._labels = np.hstack(all_labels) #横向组合成一个矩阵
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
                
    def _shuffle_data(self):
        #[0,1,2,3,4,5] ->[3,4,2,5,1.0] 在训练集上做
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data , batch_labels

    
    '''至此处理完了cifar数据'''


train_filemnames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_data_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filemnames,True)
test_data = CifarData(test_data_filenames,False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data)
print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)
[[-0.34901961 -0.54509804 -0.39607843 ... -0.00392157 -0.06666667
  -0.22352941]
 [-0.19215686 -0.21568627 -0.27843137 ...  0.4745098   0.49019608
   0.49803922]
 [ 0.45098039  0.46666667  0.56078431 ... -0.40392157 -0.41960784
  -0.39607843]
 ...
 [ 0.52156863  0.51372549  0.45882353 ... -0.15294118 -0.18431373
  -0.2       ]
 [-0.7254902  -0.7254902  -0.70980392 ... -0.56862745 -0.57647059
  -0.57647059]
 [-0.79607843 -0.56862745 -0.17647059 ... -0.22352941 -0.16862745
  -0.14509804]]
[4 6 6 0 1 4 3 5 5 4]


In [5]:
"""在tensorflow里，要先把图搭建起来，再往里面填数据。"""
tf.reset_default_graph()
x = tf.placeholder(tf.float32,[None,3072])  
#这是一个占位符，也是一个变量，可以往里面塞数据。这是data的palceholder，None表示样本数是不确定的。

y = tf.placeholder(tf.int64,[None])  
#这是labels的placeholder，因为都是离散变量，所以用int64

#（3072，10）
w = tf.get_variable('w',[x.get_shape()[-1],10],
                   initializer = tf.random_normal_initializer(0,1)) 
#w是要和x做内积的权重，w的大小就是x的大小中的3072，这是一个多分类器，所以输出为10，
#还需要定义一个初始化的方法，这里使用比较通用的正态分布。均值是1，方差为1.


#（10，）                                                                    
b = tf.get_variable('b',[10],
                   initializer =tf.constant_initializer(0.0) )  
#b是偏置，大小为w的第二维度，也就是1，输出也是1，初始化方法使用常数初始化0.

#[None,3072]*[3072,10] = [None,10]
y_ = tf.matmul(x,w) + b 
# y_是x和w做内积加上偏置b后的结果。因为x和w在多样本下都是矩阵，所以用matmul矩阵乘法
'''
#mean square loss
p_y = tf.nn.softmax(y_)
# course: 1 + e^x
# api: e^x / sum(e^x)
#[[0.01,0.03,0.4,...,0.2],[...]] 这是labels的一个概率分布，为了与之对应要把y也变成一个概率分布，用onehot算法。
y_one_hot = tf.one_hot(y,10,dtype = tf.float32)
# 5-->[0,0,0,0,1,0,0,0,0,0]
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
#同样使用平方差公式计算损失值。'''

loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)
#y_ ->softmax
#y ->one_hot
#loss = ylogy_


"""我们还对准确率感兴趣，所以现在来计算准确率"""
#indice
predict = tf.argmax(y_,1)   #取每一行中最大的数

#[1,0,1,1,1,0,1]类似的
correct_prediction = tf.equal(predict,y) 

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64)) #准确率就是我们上面所说的这个值的平均值

'''定义一个梯度下降的方法'''
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) 
    #AdamOptimizer是梯度下降的一个变种，1e-3是一个初始化的generate，并且我们优化的是loss值


In [10]:
init = tf.global_variables_initializer() 
# 构建一个初始化函数，用于初始化数据
batch_size = 20
train_steps = 100000
test_steps = 100
'''构建一个session，用于执行计算图'''
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss:%4.5f, acc:%4.5f'\
                 %(i+1,loss_val,acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_data_filenames, False)
            all_test_acc_val =[]
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Steps: %d, acc:%4.5f' %(i+1, test_acc))
#通过调用session的run函数执行计算图。可以计算loss, accuracy, train_op。加了train_op就是在训练，没加就只是在测试。
#feed_dict是我们输入的数据，x是cifar10的图片数据，y是label数据


[Train] Step: 500, loss:0.15025, acc:0.15000
[Train] Step: 1000, loss:0.10670, acc:0.45000
[Train] Step: 1500, loss:0.14846, acc:0.25000
[Train] Step: 2000, loss:0.12170, acc:0.35000
[Train] Step: 2500, loss:0.13631, acc:0.30000
[Train] Step: 3000, loss:0.14669, acc:0.25000
[Train] Step: 3500, loss:0.11757, acc:0.40000
[Train] Step: 4000, loss:0.12123, acc:0.40000
[Train] Step: 4500, loss:0.11981, acc:0.40000
[Train] Step: 5000, loss:0.13907, acc:0.30000
(10000, 3072)
(10000,)
[Test] Steps: 5000, acc:0.31400
[Train] Step: 5500, loss:0.12002, acc:0.40000
[Train] Step: 6000, loss:0.15982, acc:0.20000
[Train] Step: 6500, loss:0.10134, acc:0.45000
[Train] Step: 7000, loss:0.16628, acc:0.15000
[Train] Step: 7500, loss:0.13845, acc:0.30000
[Train] Step: 8000, loss:0.12423, acc:0.30000
[Train] Step: 8500, loss:0.14269, acc:0.25000
[Train] Step: 9000, loss:0.14246, acc:0.25000
[Train] Step: 9500, loss:0.09004, acc:0.55000
[Train] Step: 10000, loss:0.09978, acc:0.50000
(10000, 3072)
(10000,)
[T

[Train] Step: 79000, loss:0.08384, acc:0.55000
[Train] Step: 79500, loss:0.08981, acc:0.55000
[Train] Step: 80000, loss:0.08925, acc:0.55000
(10000, 3072)
(10000,)
[Test] Steps: 80000, acc:0.38300
[Train] Step: 80500, loss:0.13578, acc:0.30000
[Train] Step: 81000, loss:0.10975, acc:0.45000
[Train] Step: 81500, loss:0.09052, acc:0.55000
[Train] Step: 82000, loss:0.13648, acc:0.30000
[Train] Step: 82500, loss:0.16815, acc:0.15000
[Train] Step: 83000, loss:0.10848, acc:0.45000
[Train] Step: 83500, loss:0.11955, acc:0.40000
[Train] Step: 84000, loss:0.09706, acc:0.50000
[Train] Step: 84500, loss:0.11832, acc:0.40000
[Train] Step: 85000, loss:0.10403, acc:0.45000
(10000, 3072)
(10000,)
[Test] Steps: 85000, acc:0.39950
[Train] Step: 85500, loss:0.06000, acc:0.70000
[Train] Step: 86000, loss:0.09760, acc:0.50000
[Train] Step: 86500, loss:0.14718, acc:0.25000
[Train] Step: 87000, loss:0.11916, acc:0.40000
[Train] Step: 87500, loss:0.08228, acc:0.60000
[Train] Step: 88000, loss:0.07831, acc:0.6